In [83]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [84]:
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPool2D, Flatten, Input
from tensorflow.keras.models import Model

In [108]:
BATCH_SIZE = 1000
NUM_EPOCHS = 10
#STEPS_PER_EPOCH = 30
START_LEARNING_RATE = 0.003

In [109]:
def model_function():
    # a layer instance is callable on a tensor, and returns a tensor
    model = tf.keras.Sequential([
        Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                      data_format='channels_last',
                      activation='relu',
                      input_shape=(13, 13, 1)),
        MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPool2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(2, activation='softmax')
    ])
    
    model.compile(optimizer=tf.train.AdamOptimizer(START_LEARNING_RATE),
              loss='mse',       # mean squared error
              metrics=['mae'])  # mean absolute error
    
    return model

In [110]:
model = model_function()

In [111]:
with open('data/board_states.npy', 'rb') as boards_file, \
    open('data/winners.npy', 'rb') as winners_file:
    boards = np.load(boards_file)
    winners = np.load(winners_file)
    
    winners_cat = np.array([[1.0, 0.0] if x==1 else [0.0, 1.0] for x in winners])

    print(boards[:100000, :, :, :].shape, winners_cat[:100000, :].shape)
    
    nums = np.arange(len(winners_cat))
    np.random.shuffle(nums)
    X_train =  boards[nums[:100000],:,:,:]
    y_train = winners_cat[nums[:100000],:]
    
    X_test =  boards[nums[100000:200000],:,:,:]
    y_test = winners_cat[nums[100000:200000],:]
    

(100000, 13, 13, 1) (100000, 2)


In [112]:
model.fit(X_train,y_train, verbose=1, batch_size = BATCH_SIZE,
              epochs=NUM_EPOCHS)

Epoch 1/10
100000/100000 [==============================] - 9s 90us/step - loss: 0.2412 - mean_absolute_error: 0.4834
Epoch 2/10
100000/100000 [==============================] - 7s 73us/step - loss: 0.2294 - mean_absolute_error: 0.4600
Epoch 3/10
100000/100000 [==============================] - 8s 77us/step - loss: 0.2234 - mean_absolute_error: 0.4482
Epoch 4/10
100000/100000 [==============================] - 11s 107us/step - loss: 0.2204 - mean_absolute_error: 0.4413
Epoch 5/10
100000/100000 [==============================] - 7s 68us/step - loss: 0.2156 - mean_absolute_error: 0.4321
Epoch 6/10
100000/100000 [==============================] - 9s 92us/step - loss: 0.2118 - mean_absolute_error: 0.4259
Epoch 7/10
100000/100000 [==============================] - 13s 134us/step - loss: 0.2081 - mean_absolute_error: 0.4169
Epoch 8/10
100000/100000 [==============================] - 12s 119us/step - loss: 0.2055 - mean_absolute_error: 0.4119
Epoch 9/10
100000/100000 [========================

In [113]:
with open('data/board_states.npy', 'rb') as boards_file, \
    open('data/winners.npy', 'rb') as winners_file:
    boards = np.load(boards_file)
    winners = np.load(winners_file)
    
    #print(boards[10000:20000, :, :, :].shape, winners[100000:20000, :].shape)
    
#     evaluate = model.evaluate(boards[:10000, :, :, :], winners_cat[10000:20000, :], steps=1)
    predict = model.predict(X_test)

In [114]:
# evaluate
predict[:100]

array([[ 0.3001813 ,  0.69981873],
       [ 0.62609029,  0.37390968],
       [ 0.27214527,  0.72785473],
       [ 0.40357065,  0.59642935],
       [ 0.6465342 ,  0.3534658 ],
       [ 0.40723309,  0.59276694],
       [ 0.88738459,  0.11261541],
       [ 0.09903799,  0.90096205],
       [ 0.76009268,  0.23990725],
       [ 0.60795599,  0.3920441 ],
       [ 0.3198916 ,  0.68010843],
       [ 0.59766155,  0.40233847],
       [ 0.55590999,  0.44408998],
       [ 0.8015765 ,  0.19842348],
       [ 0.38053995,  0.61946005],
       [ 0.81577802,  0.18422197],
       [ 0.28675517,  0.71324486],
       [ 0.71685362,  0.28314635],
       [ 0.56193882,  0.43806124],
       [ 0.59942639,  0.40057358],
       [ 0.39276573,  0.60723424],
       [ 0.70586663,  0.29413334],
       [ 0.35354379,  0.6464563 ],
       [ 0.47078317,  0.52921689],
       [ 0.86628729,  0.13371277],
       [ 0.43263575,  0.56736428],
       [ 0.24781899,  0.75218099],
       [ 0.38790971,  0.61209029],
       [ 0.84327602,